In [ ]:
import pandas as pd
import numpy as np

In [ ]:
races = pd.read_csv('../data collection/races.csv')
results = pd.read_csv('../data collection/results.csv')
qualifying = pd.read_csv('../data collection/qualifying.csv')
driver_standings = pd.read_csv('../data collection/driver_standings.csv')
constructor_standings = pd.read_csv('../data collection/constructor_standings.csv')
weather = pd.read_csv('../data collection/weather.csv',sep=",")

In [ ]:
print(races.shape)
races.head()

In [ ]:
print(results.shape)
results.head()

In [ ]:
print(qualifying.shape)
qualifying.rename(columns = {'grid_position': 'grid'}, inplace = True)
qualifying.head()

In [ ]:
driver_standings.drop(['driver_points_after_race', 'driver_wins_after_race', 'driver_standings_pos_after_race'] ,axis = 1, inplace = True)

print(driver_standings.shape)
driver_standings.head()

In [ ]:
constructor_standings.drop(['constructor_points_after_race', 'constructor_wins_after_race','constructor_standings_pos_after_race' ],axis = 1, inplace = True)

print(constructor_standings.shape)
constructor_standings.head()

In [ ]:
print(weather.shape)
weather.head()

In [ ]:
df1 = pd.merge(races, weather, how='inner', on=['season', 'round', 'circuit_id']).drop(['lat', 'long','country','weather'], axis = 1)
df2 = pd.merge(df1, results, how='inner', on=['season', 'round', 'circuit_id', 'url']).drop(['url','points', 'status', 'time'], axis = 1)

df3 = pd.merge(df2, driver_standings, how='left', on=['season', 'round', 'driver']) 
df4 = pd.merge(df3, constructor_standings, how='left', on=['season', 'round', 'constructor']) #from 1958

final_df = pd.merge(df4, qualifying, how='inner', on=['season', 'round', 'grid']).drop(['driver_name', 'car'], axis = 1) #from 1983


In [ ]:
final_df.head()

In [ ]:
# calculando as idades dos pilotos

from dateutil.relativedelta import *

final_df['date'] = pd.to_datetime(final_df.date)
final_df['date_of_birth'] = pd.to_datetime(final_df.date_of_birth)
final_df['driver_age'] = final_df.apply(lambda x: relativedelta(x['date'], x['date_of_birth']).years, axis=1)
final_df.drop(['date', 'date_of_birth'], axis = 1, inplace = True)

In [ ]:
# Preenchendo/dropando os nulls

for col in ['driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points', 
            'constructor_wins' , 'constructor_standings_pos']:
    final_df[col].fillna(0, inplace = True)
    final_df[col] = final_df[col].map(lambda x: int(x))
    
final_df.dropna(inplace = True )

In [ ]:
# Convertendo para Boleano

for col in ['weather_warm', 'weather_cold','weather_dry', 'weather_wet', 'weather_cloudy']:
    final_df[col] = final_df[col].map(lambda x: bool(x))


In [ ]:
final_df = final_df[final_df['qualifying_time'] != 0]

In [ ]:
final_df[final_df['qualifying_time'] != 0]

In [ ]:
#Função para converter o tempo de qualificação da vitória em segundos
def TimeToSec(Time):
    if (':' in Time):
        t=Time.split(':')
        ts=float(t[0])*60*60+float(t[1])*60
    else:
        ts=float(Time)*60
    return ts

In [ ]:
# calculando a diference em tempo de qualificação

final_df['qualifying_time']=final_df.apply(lambda row: TimeToSec(row['qualifying_time']),axis=1)
final_df = final_df[final_df['qualifying_time'] != 0]
final_df.sort_values(['season', 'round', 'grid'], inplace = True)
final_df['qualifying_time_diff'] = final_df.groupby(['season', 'round']).qualifying_time.diff()
final_df['qualifying_time'] = final_df.groupby(['season', 'round']).qualifying_time_diff.cumsum().fillna(0)
final_df.drop('qualifying_time_diff', axis = 1, inplace = True)


In [ ]:
# get dummies

df_dum = pd.get_dummies(final_df, columns = ['circuit_id', 'nationality', 'constructor'] )

for col in df_dum.columns:
    if 'nationality' in col and df_dum[col].sum() < 140:
        df_dum.drop(col, axis = 1, inplace = True)
        
    elif 'constructor' in col and df_dum[col].sum() < 140:
        df_dum.drop(col, axis = 1, inplace = True)
        
    elif 'circuit_id' in col and df_dum[col].sum() < 70:
        df_dum.drop(col, axis = 1, inplace = True)
    
    else:
        pass

In [ ]:
df_dum.shape

In [ ]:
df_dum.to_csv('final_df.csv', index = False)